In [1]:
# import the package 
import cobra

import pandas as pd
import numpy as np
import re
pd.options.display.max_rows = 4000

from matplotlib import pyplot as plt
plt.style.use('ggplot')

In [102]:
ipfa = cobra.io.read_sbml_model("iPFA/ipfasbml.xml")

Model does not contain SBML fbc package information.
'' is not a valid SBML 'SId'.
SBML package 'layout' not supported by cobrapy, information is not parsed
SBML package 'render' not supported by cobrapy, information is not parsed
Use of FORMULA in the notes element is discouraged, use fbc:chemicalFormula instead: <Species M_C00001_c "H2O">
Use of FORMULA in the notes element is discouraged, use fbc:chemicalFormula instead: <Species M_C00002_c "ATP">
Use of FORMULA in the notes element is discouraged, use fbc:chemicalFormula instead: <Species M_C00003_c "NAD+">
Use of FORMULA in the notes element is discouraged, use fbc:chemicalFormula instead: <Species M_C00004_c "NADH">
Use of FORMULA in the notes element is discouraged, use fbc:chemicalFormula instead: <Species M_C00005_c "NADPH">
Use of FORMULA in the notes element is discouraged, use fbc:chemicalFormula instead: <Species M_C00006_c "NADP+">
Use of FORMULA in the notes element is discouraged, use fbc:chemicalFormula instead: <Speci

In [2]:
ipfal21 = cobra.io.load_json_model("FBA_Carey_models/iPfal21.json")

In [ ]:
print(ipfa.compartments)
print(ipfal21.compartments)

In [ ]:
# find all substrates of the reaction
biomets = ipfa.reactions.Biomass_rxn_c.metabolites #r_4041 = biomassreaction 8

s = list(biomets.values()) #stoichiometry of the metabolites in the reaction
m = list(biomets.keys()) # metabolite in the reaction

# print a table of metabolites and their stoichiometry

metname = [mi.name for mi in m]# names of the metabolite in the reaction

# table printing (this is very messy)

ipfa_biomass = pd.DataFrame.from_records(map(list, zip(*[m,metname,s])),columns=['ID', 'metabolite','stoichiometry'])
print(len(ipfa_biomass))
ipfa_biomass.sort_values(by='stoichiometry', ascending=False)


In [ ]:
# find all substrates of the biomass reaction
biomets = ipfal21.reactions.biomass.metabolites 

s = list(biomets.values()) #stoichiometry of the metabolites in the reaction
m = list(biomets.keys()) # ID of the metabolite in the reaction

# print a table of metabolites and their stoichiometry

metname = [mi.name for mi in m]# names of the metabolite in the reaction

# table printing (this is very messy)

biomass_ipfal = pd.DataFrame.from_records(map(list, zip(*[m,metname,s])),columns=['ID', 'metabolite','stoichiometry'])
print(len(biomass_ipfal))
biomass_ipfal.sort_values(by='stoichiometry', ascending=False)

### Find metabolites not in both models 

In [ ]:
#biomass_ipfal.metabolite[biomass_ipfal.metabolite.isin(biomass.metabolite)]
kegg_ids = []
ids = []
for i,met in enumerate(m):
    try:
        kegg = met.annotation['kegg.compound']
    except:
        kegg=''
    kegg_ids.extend(kegg)
    ids.extend([i for _ in range(len(kegg))])

def return_kegg_id(metabolite):
    try:
        kegg = metabolite.annotation['kegg.compound']
    except:
        kegg=''
    return kegg


biomass_ipfal['KEGG'] = biomass_ipfal['ID'].apply(return_kegg_id)

#### iPFA biomass exclusive metabolites

In [ ]:
ipfa_kegg = [m.id[:-2] for m in ipfa_biomass.ID]
ipfa_biomass['KEGG']=ipfa_kegg
print(len(ipfa_biomass))
print(len(biomass_ipfal))
print(len(ipfa_biomass[ipfa_biomass.KEGG.isin(kegg_ids)]))
# Venn part of included in both biomass functions
ipfa_venn = ipfa_biomass[ipfa_biomass.KEGG.isin(kegg_ids)].sort_values(by='metabolite', ascending='True')
ipfa_venn

In [ ]:
not_venn_ipfa_biomass=ipfa_biomass[~ipfa_biomass.KEGG.isin(kegg_ids)].sort_values(by='metabolite', ascending='True')
not_venn_ipfa_biomass

#### ipfal biomass of in both included 

In [ ]:
inter, ids_l, ids_i = np.intersect1d(np.asarray(kegg_ids), np.asarray(ipfa_kegg), return_indices=True)
not_included = np.asarray(ids)[~np.isin(np.arange(len(ids)), ids_l)]
venn_ipfal = biomass_ipfal.iloc[np.asarray(ids)[ids_l]].sort_values(by='metabolite', ascending='True')
venn_ipfal

In [ ]:
0.134900*(0.45* 1/175.11 *1000)**-1

In [ ]:
biomass_ipfal[~biomass_ipfal.index.isin(venn_ipfal.index)].sort_values(by='metabolite', ascending='True')

In [ ]:
not_ipfa = biomass_ipfal[~biomass_ipfal.index.isin(venn_ipfal.index)].sort_values(by='metabolite', ascending='True')

to_add=['Fe2','Fe3','L_Malate','Riboflavin','lac_L[e]','hemozoin_e','2_Octaprenyl_6_hydroxyphenol','2_Oxoglutarate','Oxidized_glutathione']
not_ipfa = not_ipfa[not_ipfa['metabolite'].isin(to_add)]
not_ipfa = not_ipfa.drop(columns=['KEGG'])
not_ipfa.columns = ['ID_y','metabolite_x','stoichiometry_x']
not_ipfa

In [ ]:
0.01/(1.63/70)

### Rewrite iPFA biomass df for ipfal

In [ ]:
# Get correct order of dfs
venn_ipfal = biomass_ipfal.iloc[np.asarray(ids)[ids_l]].copy()
ipfa_venn = ipfa_biomass[ipfa_biomass.KEGG.isin(kegg_ids)].sort_values(by='KEGG', ascending='True')
# Give ipfal single same KEGG ID as ipfa
ipfa_ID = sorted(ipfa_venn.KEGG)
venn_ipfal['ipfa_ID'] = ipfa_ID
# Rename ipfal columns, prep for merge
venn_ipfal.columns = ['ID', 'metabolite', 'stoichiometry', 'KEGG_IDS', 'KEGG']
del ipfa_ID
# merge df on single string KEGG IDs
df_merge = pd.merge(ipfa_venn, venn_ipfal, on='KEGG')
# drop not relevant columns
df_merge = df_merge.drop(columns=['ID_x','metabolite_y','stoichiometry_y','KEGG_IDS','KEGG'])
df_merge.head()

#### iPFA biomass not included metabolites analysis
Exclude lipids as those will be added with SLIMEr and find matching metabolites for iPFA biomass metabolites, which could exist in the model just not in biomass

In [ ]:
# Lipids to exclude
lipids = ['Cholesterol','Phosphatidylcholine','Phosphatidylserine','Phosphatidylethanolamine',
          'Sphingomyelin','Triacylglycerol','Cardiolipin','1,2-Diacyl-sn-glycerol',
          'Octadecanoic acid','Hexadecanoic acid', '(9Z)-Octadecenoic acid', 'Tetradecanoic acid', 'Linoleate']
# Metabolites found with their ipfal IDs
meta_nme = {'D-myo-Inositol 1,4,5-trisphosphate':'mi145p_c',
            'FMN':'fmn_c',
            'GDP-L-fucose':'gdpfuc_c',
            'GDP-mannose':'gdpmann_c',
            'Geranylgeranyl diphosphate':'ggdp_c',
            'Isopentenyl diphosphate':'ipdp_c',
            'UDP-N-acetyl-D-glucosamine':'uacgam_c',
            'UDP-glucose':'udpg_c',
            'Ubiquinone-8':'q8_c'}
# Totally missing as metabolite from ipfal
ipfal21_missing ={'5-Methyltetrahydrofolate':'C00440_c', # uses 5_10_Methylenetetrahydrofolate to directly produce Tetrahydrofolate not the intermediate step of 5-Methyltetrahydrofolate
                  'Biotin':'C00120_c', # KEGG has part of pathway documented (fatty acid biosynthesis) but not the enzymes close to biotin
                  # biotin not so important  doi: 10.1073/pnas.1800717115
                  'Methylcobalamin':'C06453_c', # Vitamine B12 maybe add https://doi.org/10.1016/j.jinorgbio.2007.01.006
                  'Protein N6-(lipoyl)lysine':'C16237_c', # could be included possibly pyruvate dehydrogenase, a-ketoglutarate dehydrogenase, grouped reaction in ipfal
                  #doi: 10.1111/j.1365-2958.2007.05592.x 
                  'UDP-D-galactose':'C00052_c' # bind for glyco-lipids/proteins should be added https://doi.org/10.1186/s12936-015-0949-z
               }
not_venn_ipfa_biomass_no_lipids = not_venn_ipfa_biomass[~not_venn_ipfa_biomass.metabolite.isin(lipids)]
not_venn_ipfa_biomass_no_lipids[~not_venn_ipfa_biomass_no_lipids.metabolite.isin(meta_nme.keys())]

In [ ]:
ipfal_found = not_venn_ipfa_biomass_no_lipids[not_venn_ipfa_biomass_no_lipids.metabolite.isin(meta_nme.keys())].copy()
ipfal_found['ID_y']=meta_nme.values()
ipfal_found = ipfal_found.drop(columns=['ID','KEGG'])
ipfal_found.columns= ['metabolite_x','stoichiometry_x','ID_y']
biomass_new = pd.concat([df_merge,ipfal_found])

In [ ]:
biomass_new.head()

#### Final biomass no lipids

In [ ]:
not_ipfa = biomass_ipfal[~biomass_ipfal.index.isin(venn_ipfal.index)].sort_values(by='metabolite', ascending='True')

to_add=['Fe2','Fe3','L_Malate','Riboflavin','lac_L[e]','hemozoin_e','2_Octaprenyl_6_hydroxyphenol','2_Oxoglutarate','Oxidized_glutathione']
not_ipfa = not_ipfa[not_ipfa['metabolite'].isin(to_add)]
not_ipfa = not_ipfa.drop(columns=['KEGG'])
not_ipfa.columns = ['ID_y','metabolite_x','stoichiometry_x']
total_biomass = pd.concat([not_ipfa,biomass_new])
total_biomass = total_biomass.drop(columns='metabolite_x')

#### Biomass reaction writer

In [ ]:
mini_addition = dict(zip(ipfal_found['ID_y'],ipfal_found['stoichiometry_x']))
# q8_c dead end metabolite have to go thro reactions 

In [ ]:
def mke_str(item):
    try:
        return str(item.id)
    except:
        return item
biomass_ipfal['ID']= biomass_ipfal['ID'].apply(mke_str)  
meta_test = dict(zip(biomass_ipfal['ID'], biomass_ipfal['stoichiometry']))
meta_test.update(mini_addition)

In [ ]:
def mke_str(item):
    try:
        return str(item.id)
    except:
        return item
total_biomass['ID_y'] = total_biomass['ID_y'].apply(mke_str)
meta_dict = dict(zip(total_biomass['ID_y'], total_biomass['stoichiometry_x']))
meta_dict['biomass_c']=1.0

In [ ]:

with ipfal21:
    # defining new biomass reaction
    new_biomass_rec = cobra.Reaction('new_ipfa')
    new_biomass_rec.name = 'reaction follows iPFA 2017'
    ipfal21.add_reactions([new_biomass_rec])
    # Q8_ as of now dead end metabolite
    #meta_dict.pop('q8_c')
    # Dict to give Reaction metabolites and corresponding stoich factors
    ipfal21.reactions.new_ipfa.add_metabolites(meta_dict)
    
    ipfal21.objective = 'new_ipfa'
    ipfal21.reactions.biomass.knock_out()
    
    print(ipfal21.objective.expression)
    print(ipfal21.optimize())

### Understanding of metabolites and reactions

In [ ]:
ipfal21.metabolites.fe2_c #https://www.sciencedirect.com/science/article/pii/S0166685199000535

In [ ]:
ipfal21.metabolites.chsterol_c

In [ ]:
ipfal21.reactions.CHSTEROLt.annotation

#### iPFA stoichiometric value understanding

In [ ]:
ipfa.metabolites.C00157_r

In [ ]:
'''
As example the phospholipid PC:C00157_r1)))

In [ ]:
ipfal21.optimize()

In [ ]:
ipfal21.metabolites.alatrna_c.summary()

In [ ]:
ipfa.metabolites.get_by_id('C16240_c')

In [ ]:
ipfa.reactions.R07771_m

In [ ]:
ipfa.metabolites.C00725_c

In [ ]:
ipfa.reactions.R07770_m

In [ ]:
# Galactose only produced for biomass could it be work around for lactose?
ipfa.metabolites.get_by_id('C00052_c')

In [ ]:
ipfa.reactions.R00291_c

### Exchange reactions

In [ ]:
pepe_ex = ipfa.exchanges
df_ex_pepe = pd.DataFrame(pepe_ex, columns=['Reaction'])
df_ex_pepe['Metabolites'] = [list(df_ex_pepe.iloc[x][0].metabolites.keys())[0] for x in range(len(df_ex_pepe))]
df_ex_pepe['Bounds'] = [df_ex_pepe.iloc[x][0].bounds for x in range(len(df_ex_pepe))]

In [ ]:
carey_ex = ipfal21.exchanges
df_ex_carey = pd.DataFrame(carey_ex, columns=['Reaction'])
df_ex_carey['Metabolites'] = [list(df_ex_carey.iloc[x][0].metabolites.keys())[0] for x in range(len(df_ex_carey))]
df_ex_carey['Bounds'] = [df_ex_carey.iloc[x][0].bounds for x in range(len(df_ex_carey))]


In [ ]:
pepe_ex_mets = [list(pepe_ex[x].metabolites.keys())[0].id[:-2] for x in range(len(pepe_ex))]
pepe_ex_ids = [list(pepe_ex[x].metabolites.keys())[0].id for x in range(len(pepe_ex))]

ex_kegg_ids = []
ex_ids = []
ex_df = []
for i,rec in enumerate(carey_ex):
    try:
        met = list(rec.metabolites.keys())[0]
        kegg = met.annotation['kegg.compound']
    except:
        kegg=''
    ex_kegg_ids.extend(kegg)
    ex_ids.extend([i for _ in range(len(kegg))])
    ex_df.append(kegg)
#carey_ex_mets = [.name for x in range(len(carey_ex))]

In [ ]:
inter, ids_l, ids_i = np.intersect1d(np.asarray(ex_kegg_ids), np.asarray(pepe_ex_mets), return_indices=True)
inter

In [ ]:
df_ex_pepe['Name']=[df_ex_pepe.Metabolites.iloc[x].name for x in range(len(df_ex_pepe))]
df_ex_pepe.iloc[np.isin(np.arange(len(pepe_ex_ids)), ids_i)]

In [ ]:
not_included = np.asarray(ex_ids)[~np.isin(np.arange(len(ex_ids)), ids_l)]
included = np.asarray(ex_ids)[np.isin(np.arange(len(ex_ids)), ids_l)]

df_ex_carey['KEGG']=ex_df
print(len(included))
print(len(not_included))
print(len(df_ex_carey))
df_ex_carey#.iloc[included]

In [ ]:
ipbe = cobra.io.load_matlab_model('/home/karnet/Downloads/ipbeblood.mat')

In [ ]:
ipbe.metabolites.C00157_r

In [101]:
ipbe.reactions.R01321_r	

NameError: name 'ipbe' is not defined

# Functions to create SLIME reactions

In [3]:
def SLIMEr_react_dict(metabolite:str) -> dict:
    '''
    Function to produce metabolite dict for SLIME reaction of lipid species 'metabolite'
    RETURN
        meta_dict: keys = metabolite name strings, values = stoichmetric factor
    '''
    edukt = metabolite
    to_slice_start = metabolite.find('(')
    to_slice_end = metabolite.find(')')
    #o_lipid  = metabolite.find('O')
    if to_slice_start!=-1:
        metabolite = metabolite[:to_slice_start-1] + metabolite[to_slice_end+1:]
    metabolite = metabolite.replace('_', ' ')
    products = metabolite.split(' ')
    
    if 'SM' in metabolite or 'Cer' in metabolite:
        products = products + ['18:1']

    meta_dict = {product:1 for product in products}
    meta_dict[edukt]=-1

    return meta_dict

def create_species(species_info):
    #print(species_info)
    if type(species_info)!=dict:
        print('Not a dict!')
        return
    new_meta = cobra.Metabolite(**species_info, compartment='cytoplasm')
    #print(new_meta)
    return new_meta

def check_meta_exists(model, meta_id):
    return meta_id in model.metabolites

def create_reaction(model, react_name:list, meta_dict:dict, revs=False):
    if react_name[0] in model.reactions:
        print('Reaction already exists, delete it first '+ react_name[0])
        return model
    
    for meta in meta_dict.keys():
        if not check_meta_exists(model, meta):
            print('No existing metabolite ' + meta)
            return model

    # defining new reaction
    reaction = cobra.Reaction(react_name[0])
    reaction.name = 'SLIME reaction' + react_name[1]
    reaction.subsystem = 'SLIMEr reaction, Lipid metabolism'
    
    if not revs:
        reaction.lower_bound = 0.  # This is the default
    else:
        reaction.lower_bound = -1000
    reaction.upper_bound = 1000.  # This is the default 
    model.add_reactions([reaction])

    model.reactions.get_by_id(reaction.id).add_metabolites(meta_dict)
    #print(model.reactions.get_by_id(reaction.id))

    return model

In [38]:
ipfal21 = cobra.io.load_json_model("FBA_Carey_models/iPfal21.json")

KeyboardInterrupt: 

In [4]:
test = {'CE': 1, '16:0': 1, 'CE 16:0': -1}
rct_lst_str = ['SLIME_xolest160','CE 16:0']
create_reaction(ipfal21,rct_lst_str,test)

No existing metabolite CE


Name,iPfal19_v1
Memory address,7fe28e4811e0
Number of metabolites,983
Number of reactions,1233
Number of genes,475
Number of groups,0
Objective expression,1.0*biomass - 1.0*biomass_reverse_01e59
Compartments,"c, e, ap, m, fv"


In [4]:
ipfal21.compartments

{'cytoplasm': 'c',
 'extracellular': 'e',
 'apicoplast': 'ap',
 'mitochondrion': 'm',
 'food_vacuole': 'fv'}

In [5]:
species_name_dict={'CE': ['xolest_bkb','cholesterol-ester'],
                    'PC':['pc_bkb','Phosphatidylcholine'],
                    'PE':['pe_bkb','Phosphatidylethanolamine'],
                    'PS':['ps_bkb','Phosphatidylserine'],
                    'PG':['pg_bkb','Phosphatidylglycerol'],
                    'SM':['sphmyln_bkb','Sphingomyelin'],
                    'FreeChol':['chsterol_bkb','Cholesterol'],
                    'TAG':['tag_bkb','Triacylglycerol'],
                    'DAG':['12dgr_bkb','1,2-Diacyl-sn-glycerol'],
                    'Cer':['cer_bkb','Ceramide'],
                    'DHSM':['dhsphmyln_bkb','Dihydrosphingomyelin'],
                    'pseudo head':['pseudo_h_bkb','Pseudo head lipid']}

for species in species_name_dict:
    sid = species_name_dict[species][0]
    sname = species_name_dict[species][1]+' backbone' 
    new_meta = cobra.Metabolite(id=sid,name=sname, compartment='cytoplasm')
    ipfal21.add_metabolites([new_meta])

In [6]:
metas_df = pd.read_csv('Datasets/slimeR_pseudo_all_percent.tsv',sep='\t',index_col=0)
metas_df = metas_df.drop('LPC')

In [7]:
meta_ids = []
for meta in metas_df.index:
    if meta[0].isdigit():
        sid = 'c'+meta.replace(':','')+'_chain'
        sname = 'Side chain '+meta 
        # add pseudo side chain information to species dict
        species_name_dict[meta]=[sid,sname]

        new_meta = cobra.Metabolite(id=sid,name=sname, compartment='cytoplasm')
        ipfal21.add_metabolites([new_meta])
    else:
        sid = species_name_dict[meta][0]
        sname = species_name_dict[meta][1]+' backbone' 
        new_meta = cobra.Metabolite(id=sid,name=sname, compartment='cytoplasm')
        ipfal21.add_metabolites([new_meta])
    meta_ids.append(sid)
metas_df['ids'] = meta_ids

### Pseudo reaction head group

In [8]:
# decide stage to implement
stage = 'mean_Ring '

{'cytoplasm': 'c',
 'extracellular': 'e',
 'apicoplast': 'ap',
 'mitochondrion': 'm',
 'food_vacuole': 'fv'}

### Pseudo reaction tails

In [9]:
sid = 'pseudo_sidchn'
sname = 'Pseudo side chain'
new_meta = cobra.Metabolite(id=sid,name=sname, compartment='cytoplasm')
ipfal21.add_metabolites([new_meta])

b2 = metas_df[metas_df.reaction=="2b"]
# change between different models
b2 = b2.dropna(subset=[stage])
tester = dict(zip(b2['ids'],b2[stage]*-1))
tester[sid] = 1
create_reaction(ipfal21,['slimer_2b','pseudo sid chain'], tester)

Name,iPfal19_v1
Memory address,7fdf231d3ac0
Number of metabolites,1034
Number of reactions,1234
Number of genes,475
Number of groups,0
Objective expression,1.0*biomass - 1.0*biomass_reverse_01e59
Compartments,"c, e, ap, m, fv"


In [10]:
metas_df = pd.read_csv('Datasets/slimeR_PLs_MW.tsv',index_col=0)
#metas_df = metas_df.drop(columns=metas_df.columns[-2:])
metas_df = metas_df.dropna(subset=[stage])
metas_df['MW'] = metas_df['MW'].apply(lambda x : float(x.replace(',','.')))
#preselection only lipids of interest if positive value
metas_df[stage] = metas_df[stage].apply(lambda x : float(x.replace(',','.')))
metas_df = metas_df[metas_df[stage]>=1e-9]

In [11]:
def crt_species_dict(df_row, loc)-> dict:
    species_dict = {}
    species_dict['id']=df_row.Model_id+'_'+loc
    species_dict['formula']=df_row.Formula
    species_dict['name']=df_row.Name

    return species_dict

for meta in metas_df.index:
    meta_dict = crt_species_dict(metas_df.loc[meta],'c')
    
    #meta_dict['annotation']={'MW':metas_df.loc[meta].MW,'metanetx.chemical':metas_df.loc[meta].MetNetX}
    species_name_dict[meta] = [meta_dict['id'],meta_dict['name']]
    ipfal21.add_metabolites([create_species(meta_dict)])

    if metas_df.loc[meta].reaction=='T':
        meta_dict = crt_species_dict(metas_df.loc[meta],'e')
        ipfal21.add_metabolites([create_species(meta_dict)])

In [12]:
for meta in metas_df.index:
    react_id = 'slimer_spl_'+metas_df.loc[meta].Model_id
    react_nme = 'Reaction to split '+metas_df.loc[meta].Name
    react_dict = SLIMEr_react_dict(meta)
    id_dict ={species_name_dict[k][0]:v for k,v in react_dict.items()}
    create_reaction(ipfal21,[react_id,react_nme], id_dict)

In [13]:
def transport_react(model, start_loc, end_loc, meta_id, meta_name, revs=True):
    
    mid = meta_id
    react_id = 'trsprt_'+mid+'_'+start_loc+'_'+end_loc
    react_nme = 'Transport reaction of '+meta_name+' from '+start_loc+' to '+end_loc
    
    mids = mid+'_'+start_loc
    mide = mid+'_'+end_loc

    id_dict = {mids:-1,mide:1}
    for key in id_dict.keys():
        if not check_meta_exists(model,key):
            print('create species '+key)

    create_reaction(model,[react_id,react_nme], id_dict,revs)

    return model

In [14]:
for meta in metas_df[metas_df.reaction =='T'].index:
    transport_react(ipfal21, 'e', 'c', metas_df.loc[meta].Model_id, metas_df.loc[meta].Name)

Tasks:

- change cholesterol import
- lipid production correct dag
    - production of correct dag
- fatty acid synthesis?
- lysoPC? 

In [15]:
# Regular expression pattern, search for all side chain configurations
pattern = r'(\d+:\d+)_(\d+:\d+)'
strings = metas_df[metas_df.reaction !='T'].index
# Find all occurrences of the pattern in each string
matches = [re.findall(pattern, s) for s in strings]
# 1-D list only with set entries 
matches = set([match[0] for match in matches if match])
# produce id as string for 1-D list        
dags_id_to_build = [f"12dgr{match[0]}{match[1]}_c".replace(':','') for match in matches]
# produce name as string for 1-D list        
dag_names = [f'1,2-Diacyl-sn-glycerol {match[0]} {match[1]}' for match in matches]

In [16]:
dag_mets = []
for i,dag_id in enumerate(dags_id_to_build):
    if check_meta_exists(ipfal21, dag_id):
        continue
    else:
        dag_mets.append(cobra.Metabolite(id=dags_id_to_build[i], name=dag_names[i], compartment='cytosol'))
ipfal21.add_metabolites(dag_mets)

### Produce DAG from PA
https://www.nature.com/articles/s41420-019-0187-1

In [69]:
def small_create_species(model, sid, sname, side_chains, exchange=False, excomp='c'):
    model_comp = {'c': 'cytoplasm',
                  'e': 'extracellular',
                  'ap': 'apicoplast',
                  'm': 'mitochondrion',
                  'fv': 'food_vacuole'}
    
    side_chains = side_chains.replace(':','')
    sid =sid+side_chains.replace('_','')+'_c'
    sname = sname+' '+side_chains
    new_meta = cobra.Metabolite(id=sid,name=sname, compartment='cytoplasm')
    model.add_metabolites([new_meta])

    if exchange:
        new_meta = cobra.Metabolite(id=sid[:-1]+excomp,name=sname, compartment=model_comp[excomp])
        model.add_metabolites([new_meta])
        model = transport_react(model, 'e', 'c', sid[:-2], sname, revs=True)

    return model, sid

def crt_reaction(model, metas_dict, react_dict, gpr):
    
    if react_dict['id'] in model.reactions:
        print('Reaction already exists, delete it first '+ react_dict['id'])
        return model
    
    # create reaction
    react_dict = cobra.Reaction(**react_dict)
    # add reaction to model
    model.add_reactions([react_dict])
    # connect to metabolites
    model.reactions.get_by_id(react_dict.id).add_metabolites(metas_dict)
    # connect to gene
    model.reactions.get_by_id(react_dict.id).gene_reaction_rule = gpr
    
    return model

react_dict = {'id':'PAP2',
              'name':'1,2-diacyl-sn-glycerol 3-phosphate phosphohydrolase',
              'metabolites':{'h2o_c':-1,
                             'pi_c':1},
              'gene_reaction_rule':'PF3D7_0805600',
              "lower_bound":0.0,
              "upper_bound":1000.0,
              "subsystem":"Lipids PhosphatidyletanolaminePhosphatidylserineMetabolism"
              }
metas_dict = react_dict.pop('metabolites')
gpr = react_dict.pop('gene_reaction_rule')
react_stock = react_dict['id'] 

for match in matches:
    side_chains = f"_{match[0]}_{match[1]}".replace(':','')
    
    ipfal21, pa = small_create_species(ipfal21,'pa','Phosphatidic acid',side_chains, exchange=True, excomp='e')
    dag_id = f"12dgr{match[0]}{match[1]}_c".replace(':','')
    # Rename to specific reaction
    react_dict['id'] = react_dict['id']+'_c'+side_chains.replace(':','')
    # Add specific metabolites
    metas_dict[dag_id]=1
    metas_dict[pa]=-1

    crt_reaction(ipfal21, metas_dict, react_dict, gpr)
    # delete for next cycle
    metas_dict.pop(dag_id)
    metas_dict.pop(pa)
    
    react_dict['id']=react_stock


Reaction already exists, delete it first trsprt_pa160181_e_c
Reaction already exists, delete it first PAP2_c_160_181
Reaction already exists, delete it first trsprt_pa140180_e_c
Reaction already exists, delete it first PAP2_c_140_180
Reaction already exists, delete it first trsprt_pa140201_e_c
Reaction already exists, delete it first PAP2_c_140_201
Reaction already exists, delete it first trsprt_pa140160_e_c
Reaction already exists, delete it first PAP2_c_140_160
Reaction already exists, delete it first trsprt_pa141180_e_c
Reaction already exists, delete it first PAP2_c_141_180
Reaction already exists, delete it first trsprt_pa181181_e_c
Reaction already exists, delete it first PAP2_c_181_181
Reaction already exists, delete it first trsprt_pa141201_e_c
Reaction already exists, delete it first PAP2_c_141_201
Reaction already exists, delete it first trsprt_pa161200_e_c
Reaction already exists, delete it first PAP2_c_161_200
Reaction already exists, delete it first trsprt_pa140140_e_c
Rea

### Produce CDP-DAG

In [18]:

react_dict = {"id":"DASYN",
              "name":"phosphatidate cytidylyltransferase",
              "metabolites":{"ctp_c":-1.0,
                             "h_c":-1.0,
                             "ppi_c":1.0},
              "lower_bound":0.0,
              "upper_bound":1000.0,
              "gene_reaction_rule":"PF3D7_1409900",
              "subsystem":"Lipids PhosphatidyletanolaminePhosphatidylserineMetabolism"}

cdp_dag = metas_df[(metas_df['reaction'] != 'T')&(metas_df.index.str.contains('PS') | metas_df.index.str.contains('PG'))]


metas_dict = react_dict.pop('metabolites')
react_stock = react_dict['id']
gpr = react_dict.pop('gene_reaction_rule')

for row in cdp_dag.T:
    side_chains = row.split(' ')[-1].replace(':','')
    print(side_chains)
    ipfal21, cdpdag_id = small_create_species(ipfal21,'cdp_12dgr','CDP-diacylglycerol',side_chains)
    ipfal21, pa = small_create_species(ipfal21,'pa','Phosphatidic acid',side_chains, exchange=True, excomp='e')
    
    # Rename to specific reaction
    react_dict['id'] = react_dict['id']+'_c'+side_chains
    # Add specific metabolites
    metas_dict[cdpdag_id]=1
    metas_dict[pa]=-1

    crt_reaction(ipfal21, metas_dict, react_dict, gpr)
    # delete for next cycle
    metas_dict.pop(cdpdag_id)
    metas_dict.pop(pa)
    react_dict['id']=react_stock

del cdp_dag

160_181
Reaction already exists, delete it first trsprt_pa160181_e_c
160_201
Reaction already exists, delete it first trsprt_pa160201_e_c
161_200
Reaction already exists, delete it first trsprt_pa161200_e_c
181_180
Reaction already exists, delete it first trsprt_pa181180_e_c
161_201
Reaction already exists, delete it first trsprt_pa161201_e_c
181_181
Reaction already exists, delete it first trsprt_pa181181_e_c


### ADD Phosphatidylcholine producing reactions

In [19]:
pcs = metas_df[(metas_df['reaction'] != 'T') & metas_df.index.str.contains('PC')]
reaction_dict = {"id":"DAGCPT",
                 "name":"diacylglycerol cholinephosphotransferase",
                 "metabolites":{"cdpchol_c":-1.0,
                                "cmp_c":1.0,
                                "h_c":1.0,
                                },
                 "lower_bound":0.0,
                 "upper_bound":1000.0,
                 "gene_reaction_rule":"PF3D7_0628300",
                 "subsystem":"Lipids PhosphatidylcholineMetabolism",
                 #"GENE ASSOCIATION":["(PFF1375c)"]
                 }
metas_dict = reaction_dict.pop('metabolites')
gpr = reaction_dict.pop('gene_reaction_rule')

for row in pcs.T:
    side_chains = row.split(' ')[-1].replace(':','')
    dag_id ='12dgr'+side_chains.replace('_','')+'_c'
    # Rename to specific reaction
    reaction_dict['id'] = 'DAG'+side_chains+'CPT'
    # Add specific metabolites
    metas_dict[dag_id]=-1
    metas_dict[pcs.loc[row].Model_id+'_c']=1
    # create reaction
    reaction = cobra.Reaction(**reaction_dict)
    # add reaction to model
    ipfal21.add_reactions([reaction])
    # connect to metabolites
    ipfal21.reactions.get_by_id(reaction.id).add_metabolites(metas_dict)
    # connect to gene
    ipfal21.reactions.get_by_id(reaction.id).gene_reaction_rule = gpr
    # delete for next cycle
    metas_dict.pop(dag_id)
    metas_dict.pop(pcs.loc[row].Model_id+'_c')

del pcs

Ignoring reaction 'DAG140_180CPT' since it already exists.
Ignoring reaction 'DAG160_160CPT' since it already exists.
Ignoring reaction 'DAG140_201CPT' since it already exists.
Ignoring reaction 'DAG141_200CPT' since it already exists.
Ignoring reaction 'DAG160_181CPT' since it already exists.
Ignoring reaction 'DAG161_180CPT' since it already exists.


In [20]:
ipfal21.reactions.DAG140_180CPT

Reaction identifier,DAG140_180CPT
Name,diacylglycerol cholinephosphotransferase
Memory address,0x7fdf21aeeb00
Stoichiometry,"2 12dgr140180_c + 2.0 cdpchol_c --> 2.0 cmp_c + 2.0 h_c + pc140180_c + pcO140180_c 2 1,2-Diacyl-sn-glycerol 14:0 18:0 + 2.0 CDPcholine --> 2.0 CMP + 2.0 H + Phosphatidyl choline (32:0) 14:0_18:0 + Phosphatidyl choline (O-32:0) 14:0_18:0"
GPR,PF3D7_0628300
Lower bound,0.0
Upper bound,1000.0


### ADD Phosphatidyl ethanolamine producing reactions

In [21]:

pes = metas_df[(metas_df['reaction'] != 'T') & metas_df.index.str.contains('PE')]
reaction_dict = {"id":"ETHAPT",
                 "name":"diacylglycerol ethanolaminephosphotransferase",
                 "metabolites":{"cdpea_c":-1.0,
                                "cmp_c":1.0,
                                "h_c":1.0,
                                },
                 "lower_bound":-1000.0,
                 "upper_bound":1000.0,
                 "gene_reaction_rule":"PF3D7_0628300",
                 "subsystem":"Lipids PhosphatidyletanolaminePhosphatidylserineMetabolism"
                 }
metas_dict = reaction_dict.pop('metabolites')
gpr = reaction_dict.pop('gene_reaction_rule')

for row in pes.T:
    side_chains = row.split(' ')[-1].replace(':','')
    dag_id ='12dgr'+side_chains.replace('_','')+'_c'
    # Rename to specific reaction
    reaction_dict['id'] = 'ETHA'+side_chains+'PT'
    # Add specific metabolites
    metas_dict[dag_id]=-1
    metas_dict[pes.loc[row].Model_id+'_c']=1
    # create reaction
    reaction = cobra.Reaction(**reaction_dict)
    # add reaction to model
    ipfal21.add_reactions([reaction])
    # connect to metabolites
    ipfal21.reactions.get_by_id(reaction.id).add_metabolites(metas_dict)
    # connect to gene
    ipfal21.reactions.get_by_id(reaction.id).gene_reaction_rule = gpr
    # delete for next cycle
    metas_dict.pop(dag_id)
    metas_dict.pop(pes.loc[row].Model_id+'_c')Met_name

del pes

### ADD TAG production
Only production pathway to avoid futile cycle, though metabolic of use at different stages

In [22]:
reaction_dict = {"id":"DAGAT_c",
                 "name":"diacylglycerol O-acyltransferase",
                 "metabolites":{
                                "coa_c":1.0,
                                },
                 "lower_bound":0.0,
                 "upper_bound":1000.0,
                 "gene_reaction_rule":"PF3D7_0322300",
                 "subsystem":"Lipids LipidProduction"
                }
metas_dict = reaction_dict.pop('metabolites')
gpr = reaction_dict.pop('gene_reaction_rule')
tag = metas_df[(metas_df['reaction'] != 'T') & metas_df.index.str.contains('TAG')]

fa_coa_dict={'140':'tdcoa',
             '160':'pmtcoa',
             '180':'stcoa',
             '200':'arachcoa',
             '141':'tdecoa',
             '161':'hdcoa',
             '181':'odecoa',
             '201':'eicoscoa',
             }
for c20 in ['arachcoa','eicoscoa']:

    sid = c20 + '_c'
    sname = 'Acetyl CoA of fatty acid '+c20 
            
    new_meta = cobra.Metabolite(id=sid,name=sname, compartment='cytoplasm')
    ipfal21.add_metabolites([new_meta])

for row in tag.T:
    side_chains = row.split(' ')[-1].replace(':','')
    fas = side_chains.split('_')Met_name
    dag_id ='12dgr'+fas[0]+fas[1]+'_c'
    acoa_id = fa_coa_dict[fas[-1]]+'_c'

    # Rename to specific reaction
    reaction_dict['id'] = 'DAG'+side_chains+'AT'
    # Add specific metabolites
    metas_dict[dag_id]=-1
    metas_dict[acoa_id]=-1
    metas_dict[tag.loc[row].Model_id+'_c']=1
    # create reaction
    reaction = cobra.Reaction(**reaction_dict)
    # add reaction to model
    ipfal21.add_reactions([reaction])
    # connect to metabolites
    ipfal21.reactions.get_by_id(reaction.id).add_metabolites(metas_dict)
    # connect to gene
    ipfal21.reactions.get_by_id(reaction.id).gene_reaction_rule = gpr
    # delete for next cycle
    metas_dict.pop(dag_id)
    metas_dict.pop(acoa_id)
    metas_dict.pop(tag.loc[row].Model_id+'_c')

### Reactions between PS to PE

In [23]:
reaction_dict_PSD = {"id":"PSD",
                 "name":"Phosphatidylserine decarboxylase",
                 "metabolites":{"co2_c":1.0,
                                "h_c":-1.0,
                                },
                 "lower_bound":0.0,
                 "upper_bound":1000.0,
                 "gene_reaction_rule":"PF3D7_0927900",
                 "subsystem":"Lipids PhosphatidyletanolaminePhosphatidylserineMetabolism"
                 }


reaction_dict_PSSbe = {"id":"PSSbe",
                       "name":"L-serine-phosphatidylethanolamine phosphatidyltransferase, putative",
                       "metabolites":{"ser__L_c":-1.0,
                                      "etha_c":1.0,
                                      },
                        "lower_bound":0.0,
                        "upper_bound":1000.0,
                        "gene_reaction_rule":"PF3D7_1366800",
                        "subsystem":"Lipids PhosphatidyletanolaminePhosphatidylserineMetabolism"
                        }

pss = metas_df[(metas_df['reaction'] != 'T') & metas_df.index.str.contains('PS')]

metas_dict_PSD = reaction_dict_PSD.pop('metabolites')
metas_dict_PSSbe = reaction_dict_PSSbe.pop('metabolites')

gpr1 = reaction_dict_PSD.pop('gene_reaction_rule')
gpr2 = reaction_dict_PSSbe.pop('gene_reaction_rule')

react_stock = reaction_dict_PSD['id']
react_stock2 = reaction_dict_PSSbe['id']

for row in pss.T:

    side_chains = row.split(' ')[-1].replace(':','')
    pe_id ='pe'+side_chains.replace('_','')+'_c'
    # Rename to specific reaction
    reaction_dict_PSD['id'] = reaction_dict_PSD['id']+'_c'+side_chains
    # Add specific metabolites
    metas_dict_PSD[pe_id]=1
    metas_dict_PSD[pss.loc[row].Model_id+'_c']=-1

    crt_reaction(ipfal21, metas_dict_PSD, reaction_dict_PSD, gpr1)Met_name
    # delete for next cycle
    metas_dict_PSD.pop(pe_id)
    metas_dict_PSD.pop(pss.loc[row].Model_id+'_c')
    reaction_dict_PSD['id']=react_stock


    # Add specific metabolites
    metas_dict_PSSbe[pe_id] = -1
    metas_dict_PSSbe[pss.loc[row].Model_id+'_c']=1
    # Rename to specific reaction
    reaction_dict_PSSbe['id'] = reaction_dict_PSD['id']+'_c'+side_chains
    crt_reaction(ipfal21, metas_dict_PSSbe, reaction_dict_PSSbe, gpr2)
    # delete for next cycle
    metas_dict_PSSbe.pop(pe_id)
    metas_dict_PSSbe.pop(pss.loc[row].Model_id+'_c')
    reaction_dict_PSSbe['id']=react_stock2

del pss

Ignoring reaction 'PSD_c160_201' since it already exists.
Ignoring reaction 'PSD_c161_200' since it already exists.
Ignoring reaction 'PSD_c181_180' since it already exists.
Ignoring reaction 'PSD_c161_201' since it already exists.
Ignoring reaction 'PSD_c181_181' since it already exists.


### Reaction producing PS from CDP-DAG

In [24]:
reaction_dict_PSSA = {"id":"PSSA",
                     "name":"CDP-diacylglycerol-serine O-phosphatidyltransferase",
                     "metabolites":{"cmp_c":1.0,
                                    "h_c":1.0,
                                    "ser__L_c":-1.0},
                     "lower_bound":0.0,
                     "upper_bound":1000.0,
                     "gene_reaction_rule":"PF3D7_1366800",
                     "subsystem":"Lipids PhosphatidyletanolaminePhosphatidylserineMetabolism"
                     }

pss = metas_df[(metas_df['reaction'] != 'T') & metas_df.index.str.contains('PS')]

metas_dict_PSSA = reaction_dict_PSSA.pop('metabolites')

react_stock = reaction_dict_PSSA['id']
gpr = reaction_dict_PSSA.pop('gene_reaction_rule')
for row in pss.T:
    side_chains = row.split(' ')[-1].replace(':','')
    dag_id ='cdp_12dgr'+side_chains.replace('_','')+'_c'
    # Rename to specific reaction
    reaction_dict_PSSA['id'] = reaction_dict_PSSA['id']+'_c'+side_chains
    # Add specific metabolites
    metas_dict_PSSA[dag_id]=-1
    metas_dict_PSSA[pss.loc[row].Model_id+'_c']=1

    crt_reaction(ipfal21, metas_dict_PSSA, reaction_dict_PSSA, gpr)
    # delete for next cycle
    metas_dict_PSSA.pop(dag_id)
    metas_dict_PSSA.pop(pss.loc[row].Model_id+'_c')
    reaction_dict_PSSA['id']=react_stock

del pss   

### Reaction PG using CDP-DAG

In [25]:
reaction_dict = {"id":"PGSA",
                 "name":"CDP-diacylglycerol-glycerol-3-phosphate 3-phosphatidyltransferase",
                 "metabolites":{"cmp_c":1.0,
                                "glyc3p_c":-1.0,
                                "h_c":1.0},
                 "lower_bound":0.0,
                 "upper_bound":1000.0,
                 "gene_reaction_rule":"PF3D7_0820200",
                 "subsystem":"Lipids PhosphatidyletanolaminePhosphatidylserineMetabolism"}


pgs = metas_df[(metas_df['reaction'] != 'T') & metas_df.index.str.contains('PG')]

metas_dict = reaction_dict.pop('metabolites')
react_stock = reaction_dict['id']
gpr = reaction_dict.pop('gene_reaction_rule')

for row in pgs.T:
    side_chains = row.split(' ')[-1].replace(':','')
    dag_id ='cdp_12dgr'+side_chains.replace('_','')+'_c'
    # Rename to specific reaction
    reaction_dict['id'] = reaction_dict['id']+'_c'+side_chains
    # Add specific metabolites
    metas_dict[dag_id]=-1
    metas_dict[pgs.loc[row].Model_id+'_c']=1

    crt_reaction(ipfal21, metas_dict, reaction_dict, gpr)
    # delete for next cycle
    metas_dict.pop(dag_id)
    metas_dict.pop(pgs.loc[row].Model_id+'_c')
    reaction_dict['id']=react_stock

del pgs

### SM production

In [ ]:
'''
Not include as lack of evidence
{"id":"2_7_8_3",
"name":"2_7_8_3",
"metabolites":{"cdpchol_c":-1.0,
               "cmp_c":1.0,
               "crm_c":-1.0,
               "h_c":-1.0,
               "sphmyln_c":1.0},
"lower_bound":0.0,
"upper_bound":1000.0,
"gene_reaction_rule":"PF3D7_0625100 or PF3D7_0625000",
"subsystem":"Lipids SphingomyelinCeramideMetabolism",
"notes":{"NOTES":"Added second gene; no experimental evidence for using cpdchol as a substrate (also uncommon throughout eukaryotes), thus, low confidence",
         "REFERENCE":"PlasmoDB; MPMP"},
"annotation":{"CURATION":"DOI: 10.1186/s12864-017-3905-1",
              "NOTES":["Added second gene; no experimental evidence for using cpdchol as a substrate (also uncommon throughout eukaryotes), thus, low confidence"],
              "REFERENCE":["PlasmoDB; MPMP"],
              "bigg.metabolite":["2_7_8_3"],
              "ec-code":"2.7.8.3",
              "sbo":"SBO:0000176"}
}
'''

{"id":"SMS",
"name":"SMS",
"metabolites":{"crm_c":-1.0,
               "dag_c":1.0,
               "pc_c":-1.0,
               "sphmyln_c":1.0},
"lower_bound":0.0,
"upper_bound":1000.0,
"gene_reaction_rule":"PF3D7_0625100 or PF3D7_0625000",
"subsystem":"Lipids SphingomyelinCeramideMetabolism",
"notes":{"NOTES":"Added second gene",
         "REFERENCE":"PlasmoDB; MPMP"
},
"annotation":{"CURATION":"DOI: 10.1186/s12864-017-3905-1",
              "REFERENCE":"PlasmoDB; MPMP",
              "bigg.reaction":["SMS"],
              "ec-code":"2.7.8.27",
              "metanetx.reaction":["MNXR104453"],
              "sbo":"SBO:0000176"}
}

{"id":"EX_sphmyln_e",
"name":"SM_ex",
"metabolites":{"sphmyln_e":-1.0},
"lower_bound":-1000.0,
"upper_bound":1000.0,
"gene_reaction_rule":"",
"subsystem":"Exchange",
"notes":{"NOTES":"Added",
         "REFERENCE":"DOI: 10.1016/S0166-6851(00)00336-4"},
"annotation":{"CURATION":"DOI: 10.1186/s12864-017-3905-1",
              "NOTES":["Added"],
              "REFERENCE":["DOI: 10.1016/S0166-6851(00)00336-4"],
              "bigg.metabolite":["EX_sphmyln_e"],
              "sbo":"SBO:0000627"}
}


In [45]:
dhcrd1_dict = {"id":"DHCRD1",
 "name":"DHCRD1",
 "metabolites":{#"crm_c":1.0,
                #"dhcrm_c":-1.0,
                "h2o_c":1.0,
                "h_c":1.0,
                "nadp_c":-1.0,
                "nadph_c":1.0,
                "o2_c":-0.5
                },
 "gene_reaction_rule":"",
 "subsystem":"Lipids SphingomyelinCeramideMetabolism",
 "notes":{"NOTES":"added EC, and o2, H2o",
          "REFERENCE":"MPMP"},
 "annotation":{"CURATION":"DOI: 10.1186/s12864-017-3905-1",
              "REFERENCE":"MPMP",
              "bigg.reaction":["DHCRD1"],
              "ec-code":"1.14.-.-",
              "metanetx.reaction":["MNXR97385"],
              "sbo":"SBO:0000176"}
}
dhcrd2_dict ={"id":"DHCRD2",
              "name":"DHCRD2",
              "metabolites":{#"crm_c":1.0,
                #"dhcrm_c":-1.0,
                "fad_c":-1.0,
                "fadh2_c":1.0,
                "h2o_c":1.0,
                "o2_c":-0.5},
            "lower_bound":0.0,
            "upper_bound":1000.0,
            "gene_reaction_rule":"",
            "subsystem":"Lipids SphingomyelinCeramideMetabolism",
            "notes":{"NOTES":"added EC, and o2, H2o",
                    "REFERENCE":"MPMP"
                    },
            "annotation":{"CURATION":"DOI: 10.1186/s12864-017-3905-1",
                        "REFERENCE":"MPMP",
                        "bigg.reaction":["DHCRD2"],
                        "ec-code":"1.14.-.-",
                        "metanetx.reaction":["MNXR97387"],
                        "sbo":"SBO:0000176"
                        }
}
dhcer_dsat = {"id":"DSAT",
            "name":"DSAT",
            "metabolites":{#"Rtotalcoa_c":-1.0,
                            "coa_c":1.0,
                            #"dhcrm_c":1.0,
                            "h_c":1.0,
                            "sphgn_c":-1.0},
            "lower_bound":0.0,
            "upper_bound":1000.0,
            "gene_reaction_rule":"PF3D7_0508200",
            "subsystem":"Lipids SphingomyelinCeramideMetabolism",
            "notes":{"NOTES":"added gene",
                    "REFERENCE":"MPMP"},
            "annotation":{"CURATION":"DOI: 10.1186/s12864-017-3905-1",
                        "REFERENCE":"MPMP",
                        "bigg.reaction":["DSAT"],
                        "ec-code":"2.3.1.24",
                        "metanetx.reaction":["MNXR138256"],
                        "sbo":"SBO:0000176"
                        }
            }

# dhcer_id-1 dhcer_dsat +1!
# cer_id 1
# fa_coa -1
metas_dsat = dhcer_dsat.pop('metabolites')
gpr_dsat = dhcer_dsat.pop('gene_reaction_rule')
dhcer_dsat.pop('annotation')
dhcer_dsat.pop('notes')
stock_dsat = dhcer_dsat['id']

cer_sm = metas_df[(metas_df['reaction'] != 'T')&(metas_df.index.str.contains('SM') | metas_df.index.str.contains('Cer'))]
for row in cer_sm.T:
    side_chain = row[-4:].replace(':','')
    if side_chain in fa_coa_dict.keys():
        ipfal21, cer_id = small_create_species(ipfal21,'crm','Ceramide',side_chain)
        ipfal21, dhcer_id = small_create_species(ipfal21,'dhcrm','Dihydroceramide',side_chain)
        
        met_dict = metas_dsat
        met_dict[dhcer_id]=1
        met_dict[fa_coa_dict[side_chain]]=-1
        dhcer_dsat['id']= dhcer_dsat['id']+side_chain

        crt_reaction(ipfal21, met_dict, dhcer_dsat, gpr_dsat)

        dhcer_dsat['id'] = stock_dsat

AttributeError: 'str' object has no attribute 'Met_name'

In [70]:
hssm_import = {"id":"SM_host",
               "name":"SM_host",
               "metabolites":{"sphmyln_e":-1.0,
                              "sphmyln_hs_c":1.0},
               "lower_bound":0.0,
               "upper_bound":1000.0,
               "gene_reaction_rule":"",
               "subsystem":"Transport e",
               "notes":{"NOTES":"Added",
                        "REFERENCE":"DOI: 10.1016/S0166-6851(00)00336-4"},
               "annotation":{"CURATION":"DOI: 10.1186/s12864-017-3905-1",
                             "NOTES":["Added"],
                             "REFERENCE":["DOI: 10.1016/S0166-6851(00)00336-4"],
                             "bigg.metabolite":["SM_host"],
                             "sbo":"SBO:0000185"}
               }
react_dict_deg = {"id":"SMPD3l_host",
                  "name":"sphingomyelin phosphodiesterase",
                  "metabolites":{"cholp_c":1.0,
                                 #"crm_c":1.0,
                                 "h2o_c":-1.0,
                                 "h_c":1.0,
                                 #"sphmyln_hs_c":-1.0
                                 },
                  "lower_bound":0.0,
                  "upper_bound":1000.0,
                  "gene_reaction_rule":"PF3D7_1238600",
                  "subsystem":"Lipids SphingomyelinCeramideMetabolism",
                  "notes":{"NOTES":"Added host sphingomyelin breakdown",
                           "REFERENCE":"DOI: 10.1042/bj3460671, 10.1016/S0166-6851(00)00336-4"},
                  "annotation":{"CURATION":"DOI: 10.1186/s12864-017-3905-1",
                                "NOTES":["Added host sphingomyelin breakdown"],
                                "REFERENCE":["DOI: 10.1042/bj3460671, 10.1016/S0166-6851(00)00336-4"],
                                "bigg.metabolite":["SMPD3l_host"],
                                "ec-code":"3.1.4.12",
                                "sbo":"SBO:0000176"}
                 }
metas_dict_deg = react_dict_deg.pop('metabolites')
react_stock_deg = react_dict_deg['id']
gpr_deg = react_dict_deg.pop('gene_reaction_rule')
react_dict_deg.pop('notes')
react_dict_deg.pop('annotation')

print(react_dict_deg)
for row in cer_sm.T:
    side_chain = row[-4:].replace(':','')
    if not side_chain in fa_coa_dict.keys():
        print('transport', side_chain)
        ipfal21, cer_id = small_create_species(ipfal21,'cer','Ceramide',side_chain)
        ipfal21, smhs_id = small_create_species(ipfal21,'sphmyln_hs','Sphingomyelin human serum', side_chain, exchange=True, excomp='e')
        
        # Rename to specific reaction
        react_dict_deg['id'] = react_dict_deg['id']+side_chains
        metas_dict_deg[cer_id]=1
        metas_dict_deg[smhs_id]=-1

        crt_reaction(ipfal21, metas_dict_deg, react_dict_deg, gpr_deg)
        # delete for next cycle
        metas_dict_deg.pop(cer_id)
        metas_dict_deg.pop(smhs_id)
        react_dict_deg['id']=react_stock_deg

    else:
        print(cer_sm.loc[row].Model_id)

{'id': 'SMPD3l_host', 'name': 'sphingomyelin phosphodiesterase', 'lower_bound': 0.0, 'upper_bound': 1000.0, 'subsystem': 'Lipids SphingomyelinCeramideMetabolism'}
transport 250
Reaction already exists, delete it first trsprt_sphmyln_hs250_e_c
cer181160
transport 241
Reaction already exists, delete it first trsprt_sphmyln_hs241_e_c
Reaction already exists, delete it first SMPD3l_host_161_201
transport 242
Reaction already exists, delete it first trsprt_sphmyln_hs242_e_c
Reaction already exists, delete it first SMPD3l_host_161_201


In [ ]:


for row in cdp_dag.T:
    side_chains = row.split(' ')[-1].replace(':','')
    print(side_chains)
    ipfal21, cdpdag_id = small_create_species(ipfal21,'cdp_12dgr','CDP-diacylglycerol',side_chains)
    ipfal21, pa = small_create_species(ipfal21,'pa','Phosphatidic acid',side_chains, exchange=True, excomp='e')
    
    # Rename to specific reaction
    react_dict['id'] = react_dict['id']+'_c'+side_chains
    # Add specific metabolites
    metas_dict[cdpdag_id]=1
    metas_dict[pa]=-1

    crt_reaction(ipfal21, metas_dict, react_dict, gpr)
    # delete for next cycle
    metas_dict.pop(cdpdag_id)
    metas_dict.pop(pa)
    react_dict['id']=react_stock
